In [ ]:
import os
import json
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
# Put the Whoscored matchId here
match_id = 1933499    
# Número/Nombre de jornada
jornada_id = 'Final_esp_ing_w'

In [5]:
def extract_json_from_whoscored(match_id):
    """Extrae datos JSON incrustados desde la página de WhoScored para un match_id dado."""
    url = f'https://es.whoscored.com/Matches/{match_id}/Live'

    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # Espera para que cargue el JS
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    scripts = soup.find_all("script")

    for script in scripts:
        if script.string and "matchCentreData" in script.string:
            data_string = script.string.split('require.config.params["args"] = ')[1].split("};")[0] + "}"
            data_string = data_string.replace('matchId', '"matchId"')
            data_string = data_string.replace('matchCentreData', '"matchCentreData"')
            data_string = data_string.replace('formationIdNameMappings', '"formationIdNameMappings"')
            data_string = data_string.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')

            data = json.loads(data_string)
            return data

    raise ValueError("No se pudo extraer JSON del partido")

def process_events_json(data):
    """Normaliza el JSON de eventos y devuelve un DataFrame limpio."""
    events = data["matchCentreData"]["events"]
    df = pd.DataFrame(events)

    # Limpieza básica de columnas anidadas
    df['type'] = df['type'].astype(str).str.extract(r"'displayName': '([^']+)'")
    df['outcomeType'] = df['outcomeType'].astype(str).str.extract(r"'displayName': '([^']+)'")
    df['period'] = df['period'].astype(str).str.extract(r"'displayName': '([^']+)'")

    return df

def save_events_csv(df, match_id, output_folder="../data/partidos"):
    """Guarda el DataFrame de eventos en un CSV."""
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"{jornada_id}_eventos.csv")
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ Datos guardados en: {output_path}")

# -------------------------
# EJECUCIÓN
# -------------------------


try:
    print("🔄 Extrayendo datos...")
    data = extract_json_from_whoscored(match_id)

    # Eventos
    df_eventos = process_events_json(data)
    save_events_csv(df_eventos, match_id)

    # Jugadores
    players_home = pd.DataFrame(data["matchCentreData"]["home"]["players"])
    players_home["teamId"] = data["matchCentreData"]["home"]["teamId"]

    players_away = pd.DataFrame(data["matchCentreData"]["away"]["players"])
    players_away["teamId"] = data["matchCentreData"]["away"]["teamId"]

    players_df = pd.concat([players_home, players_away], ignore_index=True)

    os.makedirs("../data/partidos", exist_ok=True)
    players_csv_path = os.path.join("../data/partidos", f"{jornada_id}_jugadores.csv")
    players_df.to_csv(players_csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ Datos de jugadores guardados en: {players_csv_path}")

except Exception as e:
    print(f"❌ Error: {e}")


🔄 Extrayendo datos...
✅ Datos guardados en: ../data/partidos\Final_esp_ing_w_eventos.csv
✅ Datos de jugadores guardados en: ../data/partidos\Final_esp_ing_w_jugadores.csv
